# Diffusion Loader Debug Notebook

This notebook allows manual inspection and debugging of the DiffusionLoader.

## What it loads:
- AMICO-NODDI: ICVF, ISOVF, OD metrics
- DIPY DKI: FA, MD, AD, RD, MK, AK, RK
- DIPY MAPMRI: RTOP, RTAP, RTPP, QIV, MSD
- DSI Studio GQI: QA, GFA, ISO
- Auto-discovers available workflows from QSIParc directory

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# Load environment variables
load_dotenv()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 1. Configuration

Set up paths from environment variables or override manually.

In [ ]:
# Load paths from environment (or override below)
QSIPARC_PATH = os.getenv("QSIPARC_PATH")
QSIRECON_PATH = os.getenv("QSIRECON_PATH")
SESSIONS_CSV = os.getenv("SESSIONS_CSV")
ATLAS_NAME = os.getenv("ATLAS_NAME", "4S456Parcels")

# Optionally override paths here:
# QSIPARC_PATH = "/path/to/qsiparc"
# QSIRECON_PATH = "/path/to/qsirecon"
# SESSIONS_CSV = "/path/to/sessions.csv"

print("Configuration:")
print(f"  QSIPARC_PATH: {QSIPARC_PATH}")
print(f"  QSIRECON_PATH: {QSIRECON_PATH}")
print(f"  SESSIONS_CSV: {SESSIONS_CSV}")
print(f"  ATLAS_NAME: {ATLAS_NAME}")

## 2. Verify Paths Exist

In [ ]:
def check_path(path, name):
    if path is None:
        print(f"  {name}: NOT SET")
        return False
    p = Path(path)
    exists = p.exists()
    print(f"  {name}: {'EXISTS' if exists else 'MISSING'} - {p}")
    return exists

print("Path verification:")
check_path(QSIPARC_PATH, "QSIPARC_PATH")
check_path(QSIRECON_PATH, "QSIRECON_PATH")
check_path(SESSIONS_CSV, "SESSIONS_CSV")

## 3. Explore Directory Structure

In [ ]:
# Explore QSIParc directory structure
if QSIPARC_PATH:
    root = Path(QSIPARC_PATH)
    print("QSIParc Root Structure:")
    print(f"Root: {root}")
    
    if root.exists():
        # List workflow directories
        workflows = [d.name for d in root.iterdir() if d.is_dir() and d.name.startswith("qsirecon-")]
        print(f"\nDiscovered workflows: {workflows}")
        
        # Look at one workflow's structure
        if workflows:
            wf_dir = root / workflows[0]
            subjects = sorted([d.name for d in wf_dir.iterdir() if d.is_dir() and d.name.startswith("sub-")])[:5]
            print(f"\nFirst 5 subjects in {workflows[0]}: {subjects}")

In [ ]:
# Explore detailed structure of one subject/session
if QSIPARC_PATH:
    root = Path(QSIPARC_PATH)
    workflows = [d.name for d in root.iterdir() if d.is_dir() and d.name.startswith("qsirecon-")]
    
    if workflows:
        wf_dir = root / workflows[0]
        subjects = [d for d in wf_dir.iterdir() if d.is_dir() and d.name.startswith("sub-")]
        
        if subjects:
            first_sub = subjects[0]
            sessions = [d for d in first_sub.iterdir() if d.is_dir() and d.name.startswith("ses-")]
            
            if sessions:
                first_ses = sessions[0]
                print(f"Structure of {first_sub.name}/{first_ses.name}:")
                
                # Look for dwi/atlas directories
                dwi_dir = first_ses / "dwi"
                if dwi_dir.exists():
                    for atlas_dir in dwi_dir.iterdir():
                        if atlas_dir.is_dir():
                            print(f"\n  {atlas_dir.name}/:")
                            for f in sorted(atlas_dir.glob("*.tsv"))[:10]:
                                print(f"    {f.name}")

## 4. Load Sessions CSV

In [ ]:
if SESSIONS_CSV and Path(SESSIONS_CSV).exists():
    sessions = pd.read_csv(SESSIONS_CSV, dtype={"subject_code": str, "session_id": str})
    print(f"Sessions CSV loaded: {len(sessions)} rows")
    print(f"\nColumns: {list(sessions.columns)}")
    print(f"\nFirst 10 sessions:")
    display(sessions.head(10))
else:
    print("Sessions CSV not found")
    sessions = None

## 5. Initialize DiffusionLoader

In [ ]:
from neuroalign_preprocessing.loaders import DiffusionLoader

loader = DiffusionLoader(
    qsiparc_path=QSIPARC_PATH,
    qsirecon_path=QSIRECON_PATH,
    workflows=None,  # Auto-discover all workflows
    atlas_name=ATLAS_NAME,
    n_jobs=1  # Use serial for debugging
)

print(f"DiffusionLoader initialized:")
print(f"  qsiparc_path: {loader.paths.qsiparc_path}")
print(f"  qsirecon_path: {loader.paths.qsirecon_path}")
print(f"  atlas_name: {loader.paths.atlas_name}")
print(f"  Discovered workflows: {loader.workflows}")

## 6. Test Single Session Loading

Load a single session to inspect the data structure.

In [ ]:
# Pick the first session from the CSV
if sessions is not None and len(sessions) > 0:
    test_subject = sessions.iloc[0]["subject_code"]
    test_session = sessions.iloc[0]["session_id"]
    print(f"Testing with: sub-{test_subject}_ses-{test_session}")
else:
    # Manual override if no sessions CSV
    test_subject = "001"
    test_session = "01"
    print(f"Using manual test subject: sub-{test_subject}_ses-{test_session}")

In [ ]:
# Check which workflows have data for this session
print(f"Session directory check for sub-{test_subject}_ses-{test_session}:")
for wf in loader.workflows:
    session_dir = loader.get_session_directory(test_subject, test_session, wf)
    if session_dir:
        n_files = len(list(session_dir.glob("*_parc.tsv")))
        print(f"  {wf}: EXISTS ({n_files} TSV files)")
    else:
        print(f"  {wf}: NOT FOUND")

In [ ]:
# Inspect TSV files in one workflow
if loader.workflows:
    test_workflow = loader.workflows[0]
    session_dir = loader.get_session_directory(test_subject, test_session, test_workflow)
    
    if session_dir:
        print(f"TSV files in {test_workflow}:")
        for f in sorted(session_dir.glob("*_parc.tsv")):
            print(f"  {f.name}")

In [ ]:
# Load single session (all workflows)
single_session_df = loader.load_session(
    subject=test_subject,
    session=test_session,
    workflow=None,  # Load all workflows
    include_metadata=True
)

if single_session_df is not None:
    print(f"Single session loaded: {len(single_session_df)} rows")
    print(f"\nColumns: {list(single_session_df.columns)}")
    print(f"\nWorkflows: {single_session_df['workflow'].unique().tolist()}")
    print(f"Models: {single_session_df['model'].unique().tolist()}")
    print(f"Params: {single_session_df['param'].unique().tolist()}")
else:
    print("Failed to load session")

In [ ]:
# Display sample data for each workflow/model combination
if single_session_df is not None:
    for workflow in single_session_df['workflow'].unique():
        print(f"\n{'='*60}")
        print(f"WORKFLOW: {workflow}")
        print(f"{'='*60}")
        wf_df = single_session_df[single_session_df['workflow'] == workflow]
        
        # Show model/param combinations
        combos = wf_df[['model', 'param']].drop_duplicates()
        print(f"Model/Param combinations:")
        for _, row in combos.iterrows():
            print(f"  {row['model']} - {row['param']}")
        
        # Show sample
        print(f"\nSample data:")
        display(wf_df.head(5))

## 7. Inspect Raw TSV Files

Look at the raw TSV files to verify data integrity and understand the BIDS naming.

In [ ]:
from neuroalign_preprocessing.loaders.diffusion import parse_bids_entities

# Load and display raw TSV files from first workflow
if loader.workflows:
    test_workflow = loader.workflows[0]
    session_dir = loader.get_session_directory(test_subject, test_session, test_workflow)
    
    if session_dir:
        tsv_files = sorted(session_dir.glob("*_parc.tsv"))[:5]  # First 5 files
        
        for tsv_file in tsv_files:
            print(f"\n{'='*60}")
            print(f"RAW TSV: {tsv_file.name}")
            print(f"{'='*60}")
            
            # Parse BIDS entities
            entities = parse_bids_entities(tsv_file.name)
            print(f"Parsed entities: {entities}")
            
            # Load and display
            raw_df = pd.read_csv(tsv_file, sep="\t")
            print(f"Shape: {raw_df.shape}")
            print(f"Columns: {list(raw_df.columns)}")
            display(raw_df.head(3))

## 8. Load Multiple Sessions

In [ ]:
# Load a small batch of sessions for testing
if sessions is not None:
    # Create a temporary CSV with first N sessions
    n_test = min(5, len(sessions))
    test_sessions = sessions.head(n_test)
    temp_csv = Path("/tmp/test_sessions.csv")
    test_sessions.to_csv(temp_csv, index=False)
    
    print(f"Testing batch load with {n_test} sessions...")
    
    batch_df = loader.load_sessions(
        sessions_csv=temp_csv,
        workflow=None,  # All workflows
        n_jobs=1,
        progress=True
    )
    
    print(f"\nBatch loaded: {len(batch_df)} rows")
    print(f"Unique sessions: {batch_df[['subject_code', 'session_id']].drop_duplicates().shape[0]}")

In [ ]:
# Summary statistics
if 'batch_df' in dir() and batch_df is not None:
    print("Summary by workflow/model/param:")
    summary = batch_df.groupby(['workflow', 'model', 'param']).agg({
        'label': 'nunique',
        'subject_code': 'nunique'
    }).rename(columns={'label': 'n_regions', 'subject_code': 'n_subjects'})
    display(summary)

In [ ]:
# Use the get_available_parameters method
if 'batch_df' in dir() and batch_df is not None:
    params_by_model = loader.get_available_parameters(batch_df)
    print("Available parameters by model:")
    for model, params in params_by_model.items():
        print(f"  {model}: {params}")

## 9. Data Quality Checks

In [ ]:
if 'batch_df' in dir() and batch_df is not None:
    print("Data Quality Checks:")
    print("="*60)
    
    # Check for missing values
    print("\n1. Missing values per column:")
    missing = batch_df.isnull().sum()
    missing_pct = (missing / len(batch_df) * 100).round(2)
    missing_df = pd.DataFrame({'missing': missing, 'pct': missing_pct})
    display(missing_df[missing_df['missing'] > 0])
    
    # Check for negative values in metric columns
    print("\n2. Negative values check:")
    numeric_cols = batch_df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        neg_count = (batch_df[col] < 0).sum()
        if neg_count > 0:
            print(f"  WARNING: {col} has {neg_count} negative values")
    print("  Done (no warnings = good)")
    
    # Check for duplicate entries
    print("\n3. Duplicate check:")
    dup_cols = ['subject_code', 'session_id', 'workflow', 'model', 'param', 'label']
    dups = batch_df.duplicated(subset=dup_cols, keep=False).sum()
    print(f"  Duplicate rows: {dups}")
    
    # Check region counts are consistent
    print("\n4. Region count consistency:")
    region_counts = batch_df.groupby(['subject_code', 'session_id', 'workflow', 'model', 'param'])['label'].nunique()
    print(f"  Regions per workflow/model/param:")
    display(region_counts.groupby(['workflow', 'model', 'param']).agg(['min', 'max', 'mean']))

In [ ]:
# Check value ranges for common diffusion metrics
if 'batch_df' in dir() and batch_df is not None:
    print("\n5. Value range checks for diffusion metrics:")
    
    # Expected ranges (approximate)
    expected_ranges = {
        'FA': (0, 1),      # Fractional Anisotropy
        'MD': (0, 0.005),  # Mean Diffusivity (mm^2/s)
        'ICVF': (0, 1),    # Intracellular Volume Fraction (NODDI)
        'ISOVF': (0, 1),   # Isotropic Volume Fraction (NODDI)
        'OD': (0, 1),      # Orientation Dispersion (NODDI)
    }
    
    for param, (exp_min, exp_max) in expected_ranges.items():
        param_df = batch_df[batch_df['param'] == param]
        if len(param_df) > 0 and 'mean' in param_df.columns:
            actual_min = param_df['mean'].min()
            actual_max = param_df['mean'].max()
            in_range = actual_min >= exp_min and actual_max <= exp_max
            status = "OK" if in_range else "WARNING"
            print(f"  {param}: [{actual_min:.4f}, {actual_max:.4f}] expected [{exp_min}, {exp_max}] - {status}")

## 10. Visualize Sample Data

In [ ]:
# Histogram of a diffusion metric
if 'batch_df' in dir() and batch_df is not None:
    import matplotlib.pyplot as plt
    
    # Pick a common metric (FA if available)
    for metric in ['FA', 'ICVF', 'MD']:
        metric_df = batch_df[batch_df['param'] == metric]
        if len(metric_df) > 0 and 'mean' in metric_df.columns:
            fig, ax = plt.subplots(1, 1, figsize=(10, 4))
            metric_df['mean'].hist(bins=50, ax=ax)
            ax.set_xlabel('Mean Value')
            ax.set_ylabel('Count')
            ax.set_title(f'Distribution of {metric} Regional Means')
            plt.tight_layout()
            plt.show()
            break

## 11. Test Specific Workflow Loading

In [ ]:
# Load only a specific workflow
if loader.workflows and sessions is not None:
    test_workflow = loader.workflows[0]
    print(f"Loading only workflow: {test_workflow}")
    
    n_test = min(3, len(sessions))
    test_sessions = sessions.head(n_test)
    temp_csv = Path("/tmp/test_sessions.csv")
    test_sessions.to_csv(temp_csv, index=False)
    
    workflow_df = loader.load_sessions(
        sessions_csv=temp_csv,
        workflow=test_workflow,
        n_jobs=1,
        progress=True
    )
    
    print(f"\nLoaded {len(workflow_df)} rows")
    print(f"Workflows in result: {workflow_df['workflow'].unique().tolist()}")
    print(f"Models: {workflow_df['model'].unique().tolist()}")

## 12. Debug Specific Issues

Use this section to debug specific issues you encounter.

In [ ]:
# Debug a specific subject/session
debug_subject = ""  # Fill in subject code
debug_session = ""  # Fill in session ID
debug_workflow = ""  # Fill in workflow name (optional)

if debug_subject and debug_session:
    print(f"Debugging: sub-{debug_subject}_ses-{debug_session}")
    
    # Check paths for all workflows
    for wf in loader.workflows:
        sess_dir = loader.get_session_directory(debug_subject, debug_session, wf)
        if sess_dir:
            print(f"\n{wf}: {sess_dir}")
            print(f"  Files:")
            for f in sorted(sess_dir.glob("*.tsv"))[:5]:
                print(f"    {f.name}")
    
    # Try loading
    debug_df = loader.load_session(
        debug_subject, 
        debug_session,
        workflow=debug_workflow if debug_workflow else None
    )
    if debug_df is not None:
        print(f"\nLoaded {len(debug_df)} rows")
        display(debug_df.head())
    else:
        print("\nFailed to load session data")